# Detecting Credit Card Fraud

# INTRODUCTION (text)

# UNDERSTANDING AND EXPLAINING MACHINE LEARNING ALGORITHMS (text)

# UNDERSTANDING AND CLEANING THE DATA (text and code)

In [2]:
#importing packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline

In [3]:
# reading in files 
df = pd.read_csv('datasets/bs140513_032310.csv')

### Cleaning the Data

### Visualizing the Data

# MANIPULATING THE DATA (text and code)

In [4]:
transaction_dataset = pd.read_csv('datasets/bs140513_032310.csv')

# APPLYING MACHINE LEARNING (text and code) 

In [5]:
model = linear_model 
model.fit([TrainX, TrainY])
predictions = model.predict(Validate_or_test_X)

NameError: name 'linear_model' is not defined